# Creating and Training Neural Networks

In [ ]:
import numpy as np

## MNIST Handwriting Recognition with a Neural Network
MNIST is a publically available dataset of handwritten digits 0-9 used to train simple image recognition. Each image is 28x28 or 784 pixels. This means the network cells have to have 784 different inputs and weights

<img src="files/single_cell_nn.svg"></img>

We can make a network of 10 of just 10 of these cells, one to predict each different possible output.

<img src="files/10_cell_nn.svg"></img>

## Getting the dataset
First we have to import the mnist data

In [ ]:
import os
import struct
import numpy as np

"""
All code in this cell is from https://gist.github.com/akesling/5358964
"""

def read(dataset = "training", path = "./files/MNIST_data/"):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    pyplot.show()

Let's see if we're able to read the images now

In [ ]:
def load_set(dataset="training"):
    image_generator = read(dataset)
    images = []
    while True:
        try:
            image = next(image_generator)
            images.append(image)
        except StopIteration:
            break
    return images

images = load_set("training")
test_images = load_set("testing")
show(images[0][1])

In order for these images to work with our neural net design they'll need to be reshaped to a single vector instead of a matrix and converted to binary instead of 0-255

In [ ]:
DESIRED_OUTPUTS = np.zeros((len(images), 10))
for i in range(len(images)):
    DESIRED_OUTPUTS[i][images[i][0]] = 1
transformed_images = [np.greater(np.reshape(image[1],(28*28)), 0).astype("int") for image in images]
training_images = [(DESIRED_OUTPUTS[i], transformed_images[i]) for i in range(len(transformed_images))]
test_labels = [image[0] for image in test_images]
test_images = [np.greater(np.reshape(image[1],(28*28)),0).astype("int") for image in test_images]

## Setting up the neural network
Fill in the code blocks to create a functional neural network

In [ ]:
NUM_INPUTS = 28*28
LEARNING_RATE = .02

# Variables for you to work with:
# TRAINING_IMAGES: A list of (label, image) tuples where the label has shape (10,) with a 1 in the correct index
#                  and the image is a binary list of length 784
# images: If you'd prefer to get the labels numerically (1 instead of [0,1,0,...]) images is a list of (label, image) tuples,
#         just make sure not to use the image portion of this tuple.


class perceptron:
    def __init__(self):
        # *** YOUR CODE HERE ***
        self.inputs = None
        self.weights = None # It is recommended to randomly assign the weights to 0 or 1
        self.output = None
    
    def calculate(self):
        # *** YOUR CODE HERE ***
        self.output = None # You should normalize the output so that it is between 0 and 1
        return self.output

    def classify(self, image):
        # Helper method to streamline the process
        self.inputs = image
        return self.calculate()
    
    def train(self, image, expected):
        # *** YOUR CODE HERE ***
        error = 0
        self.weights += 0

# A network of 10 cells, one cell for each digit
class network:
    def __init__(self):
        self.cells = [perceptron() for i in range(10)]
    
    def train(self, image, expected):
        for i in range(10):
            self.cells[i].train(image, expected[i])
    
    def classify(self, image):
        return np.argmax([cell.classify(image) for cell in self.cells])

## Testing
Train your network on the training data and get its training and test set accuracy (the training should take approximately 1 minute)

In [ ]:
nn = network()
for image in training_images:
    nn.train(image[1], image[0])

In [ ]:
ncorrect_training = np.sum([(nn.classify(training_images[i][1]) == images[i][0]) for i in range(len(training_images))])
ncorrect_test = np.sum([(nn.classify(test_images[i]) == test_labels[i]) for i in range(len(test_images))])
print("The training accuracy is " + str(ncorrect_training/len(training_images)))
print("The test accuracy is " + str(ncorrect_test/len(test_images)))

Take a look at some of the labels your network produced for these images. Why do you think your network got the solution it did?

In [ ]:
for i in  [56952, 4905, 36314, 44236]:
    print("Image index " + str(i))
    expected = images[i][0]
    predicted = nn.classify(training_images[i][1])
    print("Your neural network classified this as a " + str(predicted) + ". The expected label was " + str(expected))
    show(images[i][1])